In this Jupyter notebook, we perform the **spherical 2-design test** (see Section 2.3 of the Master's thesis) for the **second shell** of a deep hole of the type mentioned in the notebook title.

According to Section 4.4.2 of the Master's thesis (or the accompanying Jupyter notebook that computes the distances from the first three shells of $\Lambda_{24}$ to the deep holes), the second shell has squared distance

$$
2 + \frac{2}{h}
$$

to the deep hole, where $h$ is the **Coxeter number** associated with the deep hole.


In [1]:
from tqdm.notebook import tqdm
import numpy as np
import os

In [2]:
print(os.getcwd()) 

/home/sage


In [3]:
folder = "data_leech-shells-around-zero/"

---------------------------------------

The following functions perform the **spherical 2-design test** and display the results accordingly.

In [4]:
# P is a Matrix with the columns being the vertices of the second shell around the deep hole c
def sph_two_design_test(P,c):
    X = P.ncols()        #order of the shell around c
    n = P.nrows()        #dimension of the space
    r = (P.column(0)-c).norm()                 
    M = P -Matrix([c]*X).transpose()  #new Matrix where the vertices are P-c   
        
    #Test, whether all points have the same distance to the hole c
    for index, element in enumerate(M.columns()):
        if element.norm() != r:
            raise ValueError("Points of P do not have same distance to c "+ 
                             f" Element at index {index} is {element} and radius {element.norm()} but it should be {r}")
    
    s_1 = sum(M.columns()) # 1-design test
    s_2 = Matrix(M.columns()).transpose() * Matrix(M.columns()) 
    
    # checking codition for the spherical 2-design test
    (b_1,b_2) = (s_1 == 0, s_2 == r^2 * (X/n) * identity_matrix(n)) 
    return (b_1, b_1 & b_2, X, n, r^2, s_1, s_2, P, M, c)

#Function to present the results
def print_result(res):
    print(f"Is spherical 1-design: {res[0]}")
    print(f"Is spherical 2-design: {res[1]}")
    print("")
    print(f"Number of points of in the shell: {res[2]}")
    print(f"Dimension :  {res[3]}")
    print(f"Squared distance to the deep hole: {res[4]}")
    print("")
    print("The vector of the spherical 1-design test: ")
    print(res[5])
    print("")
    print("The matrix of the spherical 2-design test:")
    print(res[6])
    print("")
    print(f"The deep hole: ")
    print(res[9])
    return 

--------------------------------------------------------------------------------------------------

A deep hole of type $ A_8^3 $, scaled by $ \sqrt{8}\,s $, where $ s $ is the second entry in the tuple below:


In [5]:
deep_hole = (np.array(vector([27,1,1,1, 3,1,1,1, -9,5,5,5, 3,5,5,5, 9,7,7,7, 3,5,5,5])), 9)
h = 9

The procedure is as follows.

Since our database of $ \Lambda_{24} $ is scaled by $ \sqrt{8} $ so that all entries are integers, we need to scale by a factor $ s $, as our deep hole is also scaled accordingly. The squared distance from the second shell to the deep hole thus becomes the integer:

$$
(2 + \frac{2}{h}) \cdot 8 \cdot s^2,
$$

where $ h $ is the associated Coxeter number.

Since we are working exclusively with integers, we can safely use NumPy functions and still obtain exact results—this is the approach we take.  

We search through each of the first three shells for points that lie at the appropriate distance from our deep hole and store them in the list $ P$.  
At the end, we scale everything accordingly and perform the spherical 2-design test on the gathered point set.

To save RAM, we will use the data type `np.int32`, which is perfectly valid given the size of the numbers involved.

Also note that all points from the second shell of the chosen deep hole are contained in our database (see Appendix E of the Master's thesis).

In [6]:
dist_sec_shell = (2 + 2/h)*8*(deep_hole[1]^2) #

In [7]:
P = []

**First Shell**

In [8]:
np_shell_1_around_zero = np.load(folder+'shell1_around_zero.npy')
print(np_shell_1_around_zero.shape)

(196560, 24)


In [9]:
differences = np_shell_1_around_zero*deep_hole[1] - deep_hole[0] 

squared_distances = np.sum(differences.astype(np.int32) ** 2, axis=1) 

coord =  np.where(squared_distances == dist_sec_shell)[0]

P = P + [np_shell_1_around_zero[i, :] for i in coord]

In [10]:
del np_shell_1_around_zero

**Second Shell**

In [11]:
np_shell_2_around_zero = np.vstack(((np.load(folder+'shell2_shape1_around_zero.npy')).reshape(-1, 24), 
                                    np.load(folder+'shell2_shape2_around_zero.npy').reshape(-1, 24), 
                                    np.load(folder+'shell2_shape3_around_zero.npy').reshape(-1, 24), 
                                    np.load(folder+'shell2_shape4_around_zero.npy').reshape(-1, 24)))
print(np_shell_2_around_zero.shape)

(16773120, 24)


In [12]:
differences = np_shell_2_around_zero*deep_hole[1] - deep_hole[0] 

squared_distances = np.sum(differences.astype(np.int32) ** 2, axis=1) 

coord =  np.where(squared_distances == dist_sec_shell)[0]

P = P + [np_shell_2_around_zero[i, :] for i in coord]

In [13]:
del np_shell_2_around_zero

**Third Shell**

In [14]:
# Due to the size of the third shell, we need to use memory mapping to avoid loading the entire array into RAM
memmap_array1 = np.load(folder+'shell3_shape1_around_zero.npy', mmap_mode='r')
memmap_array2 = np.load(folder+'shell3_shape2_around_zero.npy', mmap_mode='r')
memmap_array3 = np.load(folder+'shell3_shape3_around_zero.npy', mmap_mode='r')
memmap_array4 = np.load(folder+'shell3_shape4_around_zero.npy', mmap_mode='r')
memmap_array5 = np.load(folder+'shell3_shape5_around_zero.npy', mmap_mode='r')
memmap_array6 = np.load(folder+'shell3_shape6_around_zero.npy', mmap_mode='r')
memmap_array7 = np.load(folder+'shell3_shape7_around_zero.npy', mmap_mode='r')
memmap_array8 = np.load(folder+'shell3_shape8_around_zero.npy', mmap_mode='r')

In [15]:
shape_arrays = [memmap_array1.reshape(-1,24), memmap_array2.reshape(-1,24), memmap_array3.reshape(-1,24), 
                memmap_array4.reshape(-1,24), memmap_array5.reshape(-1,24), memmap_array6.reshape(-1,24), 
                memmap_array7.reshape(-1,24), memmap_array8.reshape(-1,24)]

In [16]:
chunk_size = 1600000
shape_arrays[7].shape

(48, 24)

In [17]:
for array in tqdm(shape_arrays):
    for i in range(0,array.shape[0],chunk_size):  
        
        differences = array[i:i + chunk_size, :]*deep_hole[1] - deep_hole[0]         
        
        squared_distances = np.sum(differences.astype(np.int32) ** 2, axis=1)
        
        coord =  np.where(squared_distances == dist_sec_shell)[0]
        
        P = P + [(array[i:i + chunk_size, :])[k, :] for k in coord]

  0%|          | 0/8 [00:00<?, ?it/s]

In [18]:
del memmap_array1
del memmap_array2
del memmap_array3
del memmap_array4
del memmap_array5
del memmap_array6
del memmap_array7
del memmap_array8

We know conduct the spherical 2-design test.

In [19]:
P = (Matrix(QQ,P)/sqrt(8)).transpose()

In [20]:
result = sph_two_design_test(P, vector(deep_hole[0].tolist()) /(sqrt(8)*deep_hole[1]))
print_result(result)

/opt/sagemath-9.3/local/lib/python3.7/site-packages/sympy/external/importtools.py:158: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(modversion) < LooseVersion(min_module_version):


Is spherical 1-design: True
Is spherical 2-design: False

Number of points of in the shell: 27
Dimension :  24
Squared distance to the deep hole: 20/9

The vector of the spherical 1-design test: 
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

The matrix of the spherical 2-design test:
[   3/2      0      0      0   -3/4    1/4    1/4    1/4      0      0      0      0      0      0      0      0   -3/4   -1/4   -1/4   -1/4      0      0      0      0]
[     0   11/6   -1/6   -1/6   -1/4   1/12 -11/12   1/12      0   -1/3   -1/3    2/3      0   -1/3    2/3   -1/3    1/4   1/12   1/12 -11/12      0    2/3   -1/3   -1/3]
[     0   -1/6   11/6   -1/6   -1/4   1/12   1/12 -11/12      0    2/3   -1/3   -1/3      0   -1/3   -1/3    2/3    1/4 -11/12   1/12   1/12      0   -1/3    2/3   -1/3]
[     0   -1/6   -1/6   11/6   -1/4 -11/12   1/12   1/12      0   -1/3    2/3   -1/3      0    2/3   -1/3   -1/3    1/4   1/12 -11/12   1/12      0   -1/3   -1/3    2/3]
[  -3/4

--------------------------------------------------------------

---------------------------------------------------------------------------------------------